In [1]:
import os
import tensorflow as tf


In [2]:
data_dir = os.path.abspath("/Users/mika/Applied-Machine-Learning/Project/dataset/CNN_Input/train/good")
print(data_dir)  # Kontrollera att sökvägen är korrekt

/Users/mika/Applied-Machine-Learning/Project/dataset/CNN_Input/train/good


In [3]:
train_good = [f for f in os.listdir(data_dir) if f.endswith('.png')]

In [4]:
# Directory paths
train_dir = "/Users/mika/Applied-Machine-Learning/Project/dataset/CNN_Input/train"
test_dir = "/Users/mika/Applied-Machine-Learning/Project/dataset/CNN_Input/test" 

# Load train data
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    train_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'
)

# Load test data
test_data = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(150, 150),
    batch_size=32,
    label_mode='categorical'  # Categorical labels for multi-class classification
)

print(f"Class names: {test_data.class_names}")

Found 83 files belonging to 4 classes.
Found 83 files belonging to 4 classes.
Class names: ['broken_large', 'broken_small', 'contamination', 'good']


In [5]:
# Normalize the image data
def normalize_img(image, label):
    return image / 255.0, label

train_data = train_data.map(normalize_img)
test_data = test_data.map(normalize_img)

In [6]:
from tensorflow.keras import layers, models
# Build the CNN model for multi-class classification
def build_cnn_model():
    model = models.Sequential([
        layers.InputLayer(shape=(150, 150, 3)),  # Input shape (150x150x3)
        
        # Convolutional layers with max pooling
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        layers.Conv2D(128, (3, 3), activation='relu', padding='same', strides=(2, 2)),
        layers.MaxPooling2D((2, 2)),
        
        # Flatten the output and pass through dense layers
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dense(4, activation='softmax')  # 4 output units, one for each class (good, bad, contaminated, etc.)
    ])
    
    # Compile the model with sparse categorical crossentropy loss
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

# Build the CNN model
cnn_model = build_cnn_model()

In [7]:
cnn_model.fit(
    train_data,
    epochs=10,
    validation_data=test_data
)

Epoch 1/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 2s 387ms/step - accuracy: 0.2377 - loss: 1.4246 - val_accuracy: 0.2530 - val_loss: 1.3786
Epoch 2/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 326ms/step - accuracy: 0.2437 - loss: 1.3835 - val_accuracy: 0.2651 - val_loss: 1.3735
Epoch 3/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 333ms/step - accuracy: 0.3108 - loss: 1.3769 - val_accuracy: 0.6024 - val_loss: 1.3694
Epoch 4/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 349ms/step - accuracy: 0.5776 - loss: 1.3690 - val_accuracy: 0.2892 - val_loss: 1.3648
Epoch 5/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 331ms/step - accuracy: 0.3207 - loss: 1.3617 - val_accuracy: 0.2771 - val_loss: 1.3559
Epoch 6/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 281ms/step - accuracy: 0.2753 - loss: 1.3568 - val_accuracy: 0.2651 - val_loss: 1.3496
Epoch 7/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 289ms/step - accuracy: 0.2927 - loss: 1.3411 - val_accuracy: 0.3735 - val_loss: 1.3348
Epoch 8/10
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 305ms/step - accuracy: 0.4543 - loss: 1.3439 - val_accuracy: 0.5904 - val_loss:

In [8]:
test_loss, test_accuracy = cnn_model.evaluate(test_data)
print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step - accuracy: 0.5108 - loss: 1.2652
Test Loss: 1.2688190937042236
Test Accuracy: 50.60%
